In [ ]:
import re
import numpy
from shapely import wkt
import pickle
from openquake.hmtk.parsers.catalogue import CsvCatalogueParser

from osgeo import ogr
from copy import deepcopy

#from prettytable import PrettyTable

from openquake.hmtk.seismicity.selector import CatalogueSelector
from openquake.hmtk.sources.area_source import mtkAreaSource
from openquake.hazardlib.const import TRT

from oqmbt.tools.general import _get_point_list
from oqmbt.oqt_project import OQtSource

from openquake.hazardlib.geo.polygon import Polygon

In [ ]:
prj_path = "/Users/kjohnson/GEM/Regions/paisl18/project/paisl.oqmbtp"
os.environ["OQMBT_PROJECT"] = prj_path

In [2]:
shapefile_filename = '/Users/kjohnson/GEM/Regions/paisl18/data/TroubleShooting/area_16.shp'

In [3]:
idname = 'Id'
driver = ogr.GetDriverByName('ESRI Shapefile')
datasource = driver.Open(shapefile_filename, 0)
layer = datasource.GetLayer()
sources = {}
id_set = set()

In [4]:
for feature in layer:
    # Read the geometry
    geom = feature.GetGeometryRef()
    polygon = wkt.loads(geom.ExportToWkt())
    xtp, y = polygon.exterior.coords.xy
    x = ([a-360 if a>180 else a for a in xtp])
    points = _get_point_list(x, y)
    # Set the ID
    if isinstance(feature.GetField(idname), str):
        id_str = feature.GetField(idname)
    elif isinstance(feature.GetField(idname), int):
        id_str = '%d' % (feature.GetField(idname))
    else:
        raise ValueError('Unsupported source ID type')
    src = OQtSource(source_id=id_str,
            source_type='AreaSource',
            polygon=Polygon(points),
            name=id_str,
            )
    if not id_set and set(id_str):
        sources[id_str] = src
    else:
        raise ValueError('Sources with non unique ID %s' % id_str)
    #print(sources)

In [5]:
print(points)
test_poly = Polygon(points)

[<Latitude=-12.455004, Longitude=167.933559, Depth=0.0000>, <Latitude=-12.922864, Longitude=173.052492, Depth=0.0000>, <Latitude=-16.234425, Longitude=174.118927, Depth=0.0000>, <Latitude=-15.038612, Longitude=-179.762958, Depth=0.0000>, <Latitude=-15.041702, Longitude=-179.763104, Depth=0.0000>, <Latitude=-16.262489, Longitude=-178.542316, Depth=0.0000>, <Latitude=-17.071093, Longitude=-179.858647, Depth=0.0000>, <Latitude=-16.515066, Longitude=179.198525, Depth=0.0000>, <Latitude=-17.216668, Longitude=176.897271, Depth=0.0000>, <Latitude=-18.030526, Longitude=176.700822, Depth=0.0000>, <Latitude=-19.658243, Longitude=176.588566, Depth=0.0000>, <Latitude=-20.174363, Longitude=176.493491, Depth=0.0000>, <Latitude=-20.955487, Longitude=175.059899, Depth=0.0000>, <Latitude=-17.749886, Longitude=175.353746, Depth=0.0000>, <Latitude=-17.693757, Longitude=174.259247, Depth=0.0000>, <Latitude=-17.076348, Longitude=172.996363, Depth=0.0000>, <Latitude=-15.280246, Longitude=172.322825, Depth=0

In [6]:
src.polygon.wkt

'POLYGON((167.933559374 -12.4550041676, 173.052491518 -12.9228635571, 174.118926606 -16.2344251473, -179.762958074 -15.0386116985, -179.763103676 -15.0417016932, -178.542316141 -16.2624892285, -179.85864706 -17.0710925073, 179.198525317 -16.51506596, 176.897270652 -17.2166679918, 176.700822084 -18.0305263487, 176.588565758 -19.6582430626, 176.493490987 -20.1743632507, 175.059899393 -20.9554868756, 175.353746182 -17.749885536, 174.259247013 -17.6937573735, 172.996363356 -17.0763475855, 172.322825405 -15.280246384, 171.256390317 -15.1399259776, 169.937378497 -14.3260676207, 168.350771827 -14.0331556202, 167.933559374 -12.4550041676, 167.933559374 -12.4550041676))'

In [19]:
catalogue_filename = '/Users/kjohnson/GEM/Regions/paisl18/data/TroubleShooting/area16_subcata.csv'
#catalogue_filename = '/Users/kjohnson/GEM/Regions/paisl18/data/catalogues/locations/FinalCatalog/Simplified/PI_final_crustal.csv'
parser = CsvCatalogueParser(catalogue_filename)
catalogue = parser.read_file()
catalogue.sort_catalogue_chronologically()
print(len(catalogue.data['depth']))                       


Catalogue Attribute field_25 is not a recognised catalogue key
Catalogue Attribute 360_1 is not a recognised catalogue key
Catalogue Attribute field_26 is not a recognised catalogue key
Catalogue Attribute field_20 is not a recognised catalogue key
Catalogue Attribute field_24 is not a recognised catalogue key
Catalogue Attribute field_22 is not a recognised catalogue key
Catalogue Attribute field_21 is not a recognised catalogue key
Catalogue Attribute field_23 is not a recognised catalogue key
Catalogue Attribute field_27 is not a recognised catalogue key
213


In [20]:
sel = CatalogueSelector(catalogue, create_copy=True)
catalogue = sel.within_polygon(test_poly, distance = None)
print(len(catalogue.data['depth'])) 

213
